In [1]:
import pandas as pd
import plotnine as pn
from plotnine import aes, ggplot
import pathlib
from datetime import datetime
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Import Datasets
The following datasets are used:
- cpi file as supplied by zindi
- currency as supplied by zindi
- jse data as supplied by zindi
-  Monthly credit detail excel released by the reserve bank (https://www.resbank.co.za/en/home/what-we-do/statistics/releases/selected-statistics)
- fuel prices pulled from open price engin
- sabor data pulled from open price engin

In [54]:
# get directory path
path = str(pathlib.Path().cwd().parent.parent.resolve())

In [57]:
cpi = pd.read_csv(path + '/data/cpi.csv')
currency = pd.read_csv(path + '/data/currency_data.csv')
credit = pd.read_excel(path + '/data/credit.xlsx', header=1)
jse = pd.read_csv(path + '/data/jse_indices.csv')
fuel = pd.read_csv(path + '/data/fuel_df.csv').drop(['Unnamed: 0'], axis=1)
sabor = pd.read_csv(path + '/data/sabor.csv')

Change dates to datetime

In [58]:
# change month to datetime format
cpi['Date'] = pd.to_datetime(cpi['Month']).dt.strftime('%Y-%m')
currency['Date'] = pd.to_datetime(currency['Date']).dt.strftime('%Y-%m')
jse['Date'] = pd.to_datetime(jse['date']).dt.strftime('%Y-%m')
credit['date'] = pd.to_datetime(credit['Date'], format='%b, %Y')
credit['Date'] = pd.to_datetime(credit['date']).dt.strftime('%Y-%m')
fuel['Date'] = pd.to_datetime(fuel['date']).dt.strftime('%Y-%m')
sabor['Date'] = pd.to_datetime(sabor['date']).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_48410/602859498.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.


Only select dates from 2022

In [59]:
# change month to datetime format
cpi_new = cpi[cpi['Date'] > '2018']
currency_new = currency[currency['Date'] > '2018']
jse_new = jse[jse['Date'] > '2018']
credit_new= credit[credit['Date'] > '2018']
fuel_new= fuel[fuel['Date'] > '2018']
sabor_new= sabor[sabor['Date'] > '2018']

In [60]:
# Add June CPI
june_cpi = pd.DataFrame({'Category': ['Headline_CPI', 'Food and nonalcoholic beverages', 'Alcoholic beverages and tobacco', 
                                      'Clothing and footwear', 'Housing and utilities', 'Household contents and services',
                                      'Health', 'Transport', 'Communication', 'Recreation and culture', 'Education',
                                      'Restaurants and hotels ', 'Miscellaneous goods and services'], 
                         'Value': [109.8, 118.3, 110.9, 104.3, 105.4, 107.7, 110.8, 112.3, 99.6, 105.3, 110.4, 110.0, 109.6]})
june_cpi['Date'] = '2023-06'

In [61]:
cpi_columns = [['Headline_CPI', 'Food and nonalcoholic beverages', 'Alcoholic beverages and tobacco', 
                                      'Clothing and footwear', 'Housing and utilities', 'Household contents and services',
                                      'Health', 'Transport', 'Communication', 'Recreation and culture', 'Education',
                                      'Restaurants and hotels ', 'Miscellaneous goods and services']]

In [62]:
cpi_new = pd.concat([cpi_new, june_cpi])
cpi_new['Date'] = pd.to_datetime(cpi_new['Date']).dt.strftime('%Y-%m')

In the credit and national excels the commas in values need to be stripped and values turned in to intergers

In [63]:
def remove_commas_and_convert_to_float(value):
    return float(value.replace(',', ''))

In [64]:
credit_new = credit_new.drop(['Share of corporations as a % of total credit',
       'Share of corporations as a % of total loans & advances',
       'Share of households as a % of total credit',
       'Share of households as % of total loans & advances', 'Investments'], axis=1)

In [65]:
credit_columns = ['Instalment sale credit', 'Leasing finance',
       'Mortgage advances', 'Overdrafts', 'General loans and advances',
       'Credit card advances', 'Of which: Total to households',
       'Total loans and advances : Households',
       'Claims on the domestic private sector',
       'Total loans and advances (excl. investments & bills)',
       'Bills discounted', 'Instalment sale credit.1', 'Leasing finance.1',
       'Mortgage advances.1', 'Overdrafts.1', 'General loans and advances.1',
       'Credit card advances.1', 'Of which: Total to corporations',
       'Claims on the domestic private sector.1',
       'Total loans and advances : Corporations',]

In [66]:
credit_new[credit_columns] = credit_new[credit_columns].applymap(remove_commas_and_convert_to_float)


# Create Features

In some of the excel sheets we do not have up to date. We will there use the feature from 2 months ao as the feature for now (ie march is the predictor for July)

In [67]:
print(jse_new['Date'].max())
print(credit_new['Date'].max())
print(currency_new['Date'].max())
print(cpi_new['Date'].max())
print(fuel_new['Date'].max())
print(sabor_new['Date'].max())

2023-06
2023-05
2023-06
2023-06
2023-07
2023-06


In [68]:
print(jse_new['Date'].min())
print(credit_new['Date'].min())
print(currency_new['Date'].min())
print(cpi_new['Date'].min())
print(fuel_new['Date'].min())
print(sabor_new['Date'].min())

2018-01
2018-01
2018-01
2022-01
2018-01
2018-01


In [69]:
jse_new['newDate'] = (pd.to_datetime(jse_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
credit_new['newDate'] = (pd.to_datetime(credit_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
currency_new['newDate'] = (pd.to_datetime(currency_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
fuel_new['newDate'] = (pd.to_datetime(fuel_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
sabor_new['newDate'] = (pd.to_datetime(sabor['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_48410/996076500.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_48410/996076500.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [70]:
jse_new = jse_new.drop(['Date', 'date'], axis=1)
credit_new = credit_new.drop(['Date', 'date'], axis=1)
currency_new = currency_new.drop(['Date'], axis=1)
fuel_new = fuel_new.drop(['Date', 'date'], axis=1)
sabor_new = sabor_new.drop(['Date', 'date'], axis=1)

## Get mean per Month for jse and currency

In [71]:
credit_new = credit_new.groupby(['newDate']).mean().reset_index()
jse_new = jse_new.groupby(['newDate']).mean().reset_index()
currency_new = currency_new.groupby(['newDate']).mean().reset_index()
fuel_new = fuel_new.groupby(['newDate']).mean().reset_index()
sabor_new = sabor_new.groupby(['newDate']).mean().reset_index()

# Merge data together for features

In [73]:
cpi_all = cpi.drop(['Month'], axis=1).merge(jse_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(credit_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(currency_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(fuel_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(sabor_new, right_on='newDate', left_on='Date', how='left')

# Model
We will model one index at a time to see what works the best

In [74]:
from models.models import HoltWintersWrapper, ProphetWrapper
from sklearn.metrics import mean_squared_error

In [144]:
def combined_model(cpi_all, month):

    cpi_cat = ['Food and non-alcoholic beverages',
                'Alcoholic beverages and tobacco',
                'Transport','Clothing and footwear',
                'Housing and utilities',
                'Household contents and services',]
    hw_cpi_cat = ['Headline_CPI',
                'Communication',
                'Recreation and culture',
                'Restaurants and hotels ',
                'Miscellaneous goods and services',
                'Health', 'Education']

    hw_results = []
    prophet_multi = []

    # fit hw
    for cat in hw_cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        hw = HoltWintersWrapper(seasonal_periods=6)

        hw.fit(y=df[df['Date'] < month].sort_values(by='Date')['Value'].values)

        hw_results.append(hw.predict(forcast=1)[0])

    df_hw_results = pd.DataFrame({'cat':hw_cpi_cat, 'pred':hw_results})

    #fit prophet
    cpi_features = credit_new.merge(jse_new, right_on='newDate', left_on='newDate', how='left')
    cpi_features = cpi_features.merge(currency_new, right_on='newDate', left_on='newDate', how='left')
    cpi_features['newDate'] = pd.to_datetime(cpi_features['newDate']).dt.strftime('%Y-%m')

    columns_to_drop = ['Category', 'Percentage Change (From Prior Month)', 'newDate']

    for cat in cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        # reorder and drop columns
        df.insert(0, 'ds', df.pop('Date'))
        df.insert(1, 'y', df.pop('Value'))
        df = df.drop(columns_to_drop, axis=1)
        df = df[['ds', 'y']]

        # model
        prophet = ProphetWrapper(interval_width=0.8)
        prophet.fit(df[df['ds'] < month].sort_values(by='ds').sort_values(by='ds'))

        df_predict = pd.DataFrame({'ds': [month]})
        df_predict['ds'] = pd.to_datetime(df_predict['ds']).dt.strftime('%Y-%m')
        df_predict = df_predict.merge(cpi_features, left_on='ds', right_on='newDate').drop(['newDate'], axis=1)
        df_predict_columns = df.drop(['y'], axis=1).columns
        prophet_multi.append(prophet.predict(df_predict[df_predict_columns])[0])

    df_prophet_results = pd.DataFrame({'cat':cpi_cat, 'pred':prophet_multi})

    df_results = pd.concat([df_hw_results, df_prophet_results])

    return df_results

        

# Get testing predictions

In [166]:
df_results = combined_model(cpi_all=cpi_all, month='2023-07')

In [167]:
df_results.columns = ['ID', 'Value']

In [168]:
month = 'July'

In [169]:
pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

In [170]:
df_results = df_results.replace(pred_map)

In [171]:
df_results.to_csv(path + f'/submissions/multv2_{month}.csv', index=False)

In [172]:
month

'July'